# 📚 WeebCentral Manga Downloader

Download manga chapters from WeebCentral with full parallel support.

### Setup
1. Run **Cell 1** once to install dependencies + Clone Repository.
2. Run **Cell 2** — paste URL, pick chapters & format, everything downloads.
3. Optionally run **Cell 3** to zip & download to your PC.

### Output formats
| Option | Result |
|--------|--------|
| `pdf` | One PDF per chapter |
| `cbz` | CBZ archive with `ComicInfo.xml` (Kavita / Komga / CDisplayEx) |
| `images` | Raw image folder per chapter |
| `all` | PDF + CBZ + Images |

### Supported URL
```
https://weebcentral.com/series/SERIES_ID/manga-slug
```

In [ ]:
# ╔══════════════════════════════════════════════════════╗
# ║  CELL 1 — Install dependencies  (run once)          ║
# ╚══════════════════════════════════════════════════════╝
!pip install -q requests 'httpx[http2]' nest_asyncio beautifulsoup4 lxml Pillow fpdf2 tqdm
print('✅ Dependencies ready.')
!git clone https://github.com/Yui007/weebcentral_downloader

In [ ]:
# ╔══════════════════════════════════════════════════════╗
# ║  CELL 2 — Scrape, select & download                 ║
# ╚══════════════════════════════════════════════════════╝
import sys
sys.path.insert(0, '/content/weebcentral_downloader/colab')

from colab_scraper    import scrape_manga_info, scrape_chapter_list
from colab_downloader import parse_chapter_selection, download_chapters

# ── 1. URL ───────────────────────────────────────────────────────────────────
SERIES_URL = input(
    '🌐 WeebCentral series URL\n'
    '   e.g. https://weebcentral.com/series/01JCZQE.../manga-slug\n'
    '   > '
).strip()

# ── 2. Scrape ────────────────────────────────────────────────────────────────
manga_info = scrape_manga_info(SERIES_URL)
chapters   = scrape_chapter_list(SERIES_URL)
total      = len(chapters)

# ── 3. Show chapter list ─────────────────────────────────────────────────────
print(f"\n{'='*62}")
print(f"  {'#':>4}   {'Chapter':<50}  Date")
print(f"{'='*62}")
for ch in chapters:
    num  = str(ch['index']).rjust(4)
    name = ch['title'][:50].ljust(50)
    print(f"  {num}   {name}  {ch['date']}")
print(f"{'='*62}")
print(f"  Total: {total} chapters\n")

# ── 4. Chapter selection ─────────────────────────────────────────────────────
print('📥 Selection formats:')
print('   all          → every chapter')
print('   single 5     → chapter 5 only')
print('   range 1-10   → chapters 1 through 10 (inclusive)')
print('   1,5,9,15     → specific chapters')
print()

while True:
    sel_input = input(f'🎯 Select chapters (1–{total}): ').strip()
    try:
        selected = parse_chapter_selection(sel_input, total)
        print(f'   ✅ {len(selected)} chapter(s) selected.')
        break
    except ValueError as e:
        print(f'   ❌ {e}\n   Try again.')

# ── 5. Output format ─────────────────────────────────────────────────────────
print()
print('📦 Output format:')
print('   pdf    → PDF file per chapter')
print('   cbz    → CBZ with ComicInfo.xml  (Kavita / Komga / CDisplayEx)')
print('   images → Raw image folder per chapter')
print('   all    → PDF + CBZ + Images')
print()

while True:
    fmt = input('🗂  Format [pdf / cbz / images / all]: ').strip().lower()
    if fmt in ('pdf', 'cbz', 'images', 'all'):
        break
    print('   ❌ Please enter pdf, cbz, images, or all.')

# ── 6. Download ──────────────────────────────────────────────────────────────
output_dir = download_chapters(
    manga_info       = manga_info,
    chapters         = chapters,
    selected_indices = selected,
    output_format    = fmt,
    output_dir       = '/content/weebcentral_downloader/colab/manga',
)

In [ ]:
# ╔══════════════════════════════════════════════════════╗
# ║  CELL 3 — (Optional) Zip & download to your PC      ║
# ╚══════════════════════════════════════════════════════╝
import re, shutil
from google.colab import files

zip_stem = re.sub(r'[\\/:*?"<>|]', '_', manga_info['title']).strip() + '_manga'
zip_path = f'/content/weebcentral_downloader/colab/{zip_stem}.zip'

print(f'📦 Zipping {output_dir} ...')
shutil.make_archive(f'/content/weebcentral_downloader/colab/{zip_stem}', 'zip', output_dir)
print(f'✅ Ready — initiating download...')
files.download(zip_path)